https://www.youtube.com/watch?v=1b7pXC1-IbE

14:49

In [6]:
!pip install -r requirment.txt

In [2]:
import cv2
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
import urllib.request

In [3]:
IZ_ONE = ["최예나", "조유리", "김채원", "안유진", "김민주", "장원영",
           "나코", "히토미", "사쿠라", "이채연", "강혜원", "권은비"]
group_name = "아이즈원"

In [10]:
driver = webdriver.Chrome()

for member in IZ_ONE:
    driver.get("https://www.google.co.kr/imghp?hl=ko&tab=ri&ogbl")
    elem = driver.find_element_by_name("q")
    elem.send_keys(f"{group_name} {member}")
    elem.send_keys(Keys.RETURN)
    
    SCROLL_PAUSE_TIME = 1
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while 1:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height: 
            try:
                driver.find_element_by_css_selector(".mye4qd").click()
            except:
                break
        last_height = new_height
    
    images = driver.find_elements_by_css_selector(".rg_i.Q4LuWd")
    for tmp_name, img in enumerate(images):
        try:
            img.click()
            time.sleep(1)
            imgUrl = driver.find_element_by_xpath("/html/body/div[2]/c-wiz/div[3]/div[2]/div[3]/div/div/div[3]/div[2]/c-wiz/div/div[1]/div[1]/div[2]/div[1]/a/img").get_attribute("src")
            urllib.request.urlretrieve(imgUrl, f"./{member}/{tmp_name}.jpg")
        except:
            pass
        
driver.close()